In [27]:
import sys
sys.path.append("/home/user/Projects/BOSK/")
from bosk.big_picture_utils import get_patches, combine_patches,segment_patch
from bosk.utils import load_model 
from bosk.imports import *

%load_ext autoreload
%autoreload 2

image_path = Path("../Some_images/Landscape_Bosk.png")
image = Image.open(image_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
splits_per_side = 16
num_patches = splits_per_side**2
patches = get_patches(image, num_patches)

fig, axes = plt.subplots(splits_per_side, splits_per_side)
for i in range(num_patches):
    row = i // splits_per_side
    col = i % splits_per_side

    axes[row, col].imshow(patches[i])

In [29]:
model_paths = [
    "../models/segformer_1.pth",
    "../models/unet_with_mobilenet_2.pth",
    "../models/newest_deeplab.pth",
]
model_types = ["segformer", "unet_mobile", "deeplab"]
assert (
    model_paths is not None and model_types is not None
), "Enter path for loading model and model's type"

In [ ]:
transform = T.Compose([
    T.Resize((512, 512)),
    T.ToTensor()
])

for model_path, model_type in zip(model_paths, model_types):
    print(f"{model_type} model is perfoming now")
    model = load_model(model_path, model_type)
    segmented_patches = {f"{model_type}": []}
    for patch in patches:
        patch = Image.fromarray(patch)
        segmented_patches[f"{model_type}"].append(segment_patch(model, patch, model_type, transform))

    result = combine_patches(segmented_patches[f"{model_type}"])
    result_path = Path(f"../Some_images/Segmented_images/{model_type}.png")
    plt.imsave(result_path, result)
    print(f"Result is saved in {result_path}\n")

segformer model is perfoming now


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([5]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([5, 256, 1, 1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Result is saved in ../Some_images/Segmented_images/segformer.png
unet_mobile model is perfoming now
Result is saved in ../Some_images/Segmented_images/unet_mobile.png
deeplab model is perfoming now
Result is saved in ../Some_images/Segmented_images/deeplab.png
